# Introduction
This 

In [1]:
import numpy as np


In [2]:
class Network:
    
    def __init__(self, architecture, activation= 'sigmoid', learning_rate= 0.001):
        self.architecture= architecture
        self.activation= activation
        self.lr= learning_rate
        self.parameters=[]
        self.buffered_op={}
        self.prime_funcs=[]
        
        if type(self.activation) != str:
            assert len(self.activation) + 1 == len(self.architecture)
        for i in range(len(architecture) -1):
            layer_parameters= self.create_weights_and_biases(architecture[i], architecture[i + 1])
            self.parameters.append(layer_parameters)
        
        
    def create_weights_and_biases(self, in_nodes, out_nodes):
        std= np.sqrt(2.0/(in_nodes + out_nodes))
        w= np.random.normal(loc= 0, scale= std, size= (in_nodes, out_nodes))
        b= np.zeros((1, out_nodes))
        return {'weight':w , 'bias':b}
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return 0 if x<0 else x
    
    def sigmoid_prime(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    def relu_prime(self, x):
        return 1 if x>0 else 0

    def forward_pass(self, x):
        self.buffered_op['a0']= x
        for i, parameter in enumerate(self.parameters):
            w= parameter['weight']
            b= parameter['bias']
            if type(self.activation) != str:
                activation= self.activation[i]
                #need to implement the prime of the activarionfunc if activation is provided
            
            if not callable(activation):
                if lower(activation)=='sigmoid':
                    activation= self.sigmoid
                    self.prime_funcs.append(sigmoid_prime)
                    
                elif lower(activation) == 'relu':
                    activation= self.relu
                    self.prime_funcs.append(relu_prime)
                    
                else:
                    raise Exception("Sorry, only relu and sigmoid supported currently") 
            
            h= np.matmul(x, w) + b
            a= activation(h)
            self.buffered_op['h' + str(i+1)]= h
            self.buffered_op['a' + str(i+1)]= a
        return a
    
    def loss(self, y_cap, y):
        return (y - y_cap) ** 2 #MSE
    
    def loss_prime(self, y_cap, y):
        return -2 * (y - y_cap)
    
    def backpropagation(self,y_cap, y):
        last_layer= True
        layer_error= self.loss_prime( self.loss(y_cap, y))
        gradients= []
        
        for layer_idx in range(len(self.architecture) - 1, 0 , -1):
            prime_func= self.prime_funcs[layer_idx]
            h_cur= self.buffered_op['h'+str(layer_idx)]
            a_prev= self.buffered_op['a'+str(layer_idx - 1)]
            
            
            error_term= layer_error * prime_func(h_cur)
            del_w= np.matmulmul(error_term.T , a_prev)
            del_b= error_term.sum(dim= 0)
            gradients.append((del_w, del_b))
            
            #update the layer_error for nxt iter
            layer_error= np.matmul(error_term , self.parameters[layer_idx -1]['weight'].T)
            
        gradients.reverse()
        
        #updating the weights:
        for i in range(len(self.parameters)):
            self.parameters[i]['weight'] += self.learning_rate * gradients[i][0].T
            self.parameters[i]['bias'] += self.learning_rate * gradients[i][1]
            
            